# Gap filling of the dta cube stacks

## 1. Importing libraries

Libraries reference:

+ [numpy](https://numpy.org/install/)
+ [scipy](https://www.scipy.org/install.html)
+ [rasterio](https://rasterio.readthedocs.io/en/latest/)
+ [tqdm](https://github.com/tqdm/tqdm)
+ [os](https://docs.python.org/3/library/os.html)
+ [gdal](https://gdal.org/api/python.html)
+ [multiprocessing](https://docs.python.org/3/library/multiprocessing.html)
+ [functools](https://docs.python.org/3/library/functools.html)
+ [datetime](https://docs.python.org/3/library/datetime.html)
+ [time](https://docs.python.org/3/library/time.html)
+ [glob](https://docs.python.org/3/library/glob.html)

In [ ]:
import numpy as np
from scipy import interpolate
from scipy.ndimage.morphology import binary_dilation
import rasterio as r
from tqdm.notebook import tqdm
import os
import gdal
import multiprocessing as mp
from functools import partial
import datetime
import time
import glob

## 2. Changing working directory

In [ ]:
# folder where all data is stored
os.chdir(os.getcwd().rsplit('/',2)[0]+'/Data')

## 3. Defining the gap filling function

In [ ]:
def CubicSpline(ts, days, bc_t, extrap):
    # ts = time series with clouds
    # days = day entry for each ts entry. must be crescent.
    # bc_t = CubicSpline bc_type. Boundary condition type. natural
    # extrap = CubicSpline extrapolate. bool
    # result = filtered time series

    # 'true' is where there is invalid pixels
    pqa = ts==-9999

    # x1: array for the Spline function
    x1 = days[np.invert(pqa)]

    # y1: array for the Spline function
    y1 = ts[np.invert(pqa)]
    
    if len(x1)>1:
        # the spline interpolator
        spline = interpolate.CubicSpline(x1,y1, bc_type=bc_t, extrapolate=extrap)

        # x values to interpolate
        x2 = np.where(pqa==True)

        result = ts.copy()
        for index in x2:
            result[index] = spline(days[index])
            
        return result
    else:
        return [-9999]*len(ts)

## 4. Filling the gaps in the data cube stacks

In [ ]:
# creating folder to save filled cubes.
os.mkdir('./cubes/filled')

# collect cubes that need filling.
cubes = [file for file in glob.glob('./cubes/raw/*.tif') if not 'Fmask4' in file]

# iterate through the cubes to be filled.
for cube_path in cubes:
    ################
    # START PROCESSING THE CUBES
    ################
    t1 = time.time()
    print('--------------------------------------------------------------')
    print('                  PROCESSING '+cube_path)
    
    ################
    # DAYS
    ################
    days_path = f'./cubes/raw/days.{cube_path.split(".")[1].split("/")[-1]}.{cube_path.split(".")[-3]}.npy'

    days = np.load(days_path)

    days2 = []
    aux1 = datetime.datetime.strptime(days[0], '%Y-%m-%d')
    days2.append(0)
    for i in range(1, len(days)):
        aux2 = datetime.datetime.strptime(days[i], '%Y-%m-%d')
        days2.append(days2[i-1]+(aux2-aux1).days)
        aux1 = aux2

    days = np.asarray(days2)
    del days2

    ################
    # LOADING FMASK
    ################
    print('Loading mask...')

    fmask_path = cube_path.replace(cube_path.split(".")[-2], 'Fmask4')
    fmask = r.open(fmask_path).read()
    a = fmask==0
    b = fmask==1

    mask = a+b
    del fmask, a, b

    ##################
    # SAVE PATH
    ##################
    save_path = cube_path.replace('raw', 'filled')

    ################
    # LOADING CUBE
    ################
    print('Loading Cube '+cube_path+'...')

    cube = np.asarray(r.open(cube_path).read())
    cube[np.invert(mask)] = -9999

    ################
    # PROCESSING
    ################
    print('Processing...')

    n = cube.shape[0]
    series_to_process = []
    ij = []
    count = 0
    n_to_process = 1000000
    for i in tqdm(range(cube.shape[1])):
        for j in range(cube.shape[2]):
            series_to_process.append(cube[:,i,j])
            ij.append([i,j])
            count +=1

            if count == n_to_process or (i==cube.shape[1]-1 and j==cube.shape[2]-1):
                with mp.Pool(processes=mp.cpu_count()-10) as pool:
                    result_series = pool.starmap(partial(CubicSpline), [(series_to_process[k], days, 'natural', bool) for k in range(len(ij))])
                for k in range(len(ij)):
                    cube[:,ij[k][0],ij[k][1]] = result_series[k]

                count = 0
                series_to_process = []
                ij = []

    ################
    # SAVING
    ################
    print('Saving...')

    ref2 = gdal.Open(cube_path)
    in_band = ref2.GetRasterBand(1)

    gtiff_driver = gdal.GetDriverByName('GTiff')

    print('File Location: '+save_path)

    out_ds = gtiff_driver.Create(save_path, in_band.XSize, in_band.YSize, cube.shape[0], in_band.DataType, ['COMPRESSION=LZW'])
    out_ds.SetProjection(ref2.GetProjection())
    out_ds.SetGeoTransform(ref2.GetGeoTransform())

    for i in range(1, cube.shape[0]+1, 1):
        band = out_ds.GetRasterBand(i)
        band.SetNoDataValue(-9999)
        band.WriteArray(cube[i-1,:,:])
        band.FlushCache()

    out_ds = None
    ref2 = None
    t2 = time.time()
    print('Elapsed time: %.3f minutes\nDone!'%((t2-t1)/60))

print('---------------------------------------------------------\nAll done!')